# 15. 모형진단과 교차검증

과대적합: 학습데이터에는 정밀도가 높으나, 시험데이터에는 정밀도가 떨어짐
과소적합: 학습데이터의 정밀도가 목표한 정밀도보다 현저하게 낮음  



## 1. 모형진단과 교차검증의 이해

### 1-1. 과대적합 과소적합

**과대적합**
- 학습데이터에는 정밀도가 높으나 시험데이터에는 정밀도가 떨어짐
- 표본수를 늘리거나, 초모수 조정 또는 규제화 등으로 해결함

**과소적합**
- 학습데이터의 정밀도가 목표한 정밀도보다 현저하게 낮음
- 입력변수를 틀리거나 다른 모형을 선택하여 해결함

### 1-2. 모형진단

여러 후보모형 중 가장 좋은 모형을 선택하기 위한 성능비교

**학습데이터를 통해 모형 학습**
- 검증데이터에 적용하여 최적의 초모수를 선택한 후 성능비교
- 학습이나 검증에 이용한 적 없는 시험데이터의 정밀도로 비교 점검

Training set = Training set + Validation set 으로 분할

### 1-3. 초모수 결정
(=hyperparameter tuning)

학습데이터를 이용하여 모형을 학습하고 검증데이터를 통해 조절
예) SVM에서 결정해야 할 C, 커널함수의 gamma 등

$\rightarrow$ 학습데이터의 성능유지, 검증데이터의 성능도 학습데이터와 근접하게 조절

**문제점**: 고정된 자료 분할은 선택된 검증데이터에 크게 의존함. 자료의 크기가 작을 경우 더욱 두드러짐.   

### 1-4. k-분할 교차검증
(=k-fold cross validation)

1. 전체 자료를 학습데이터 + 시험데이터로 나눔
2. 학습데이터를 임의의 k개 폴드로 분할함
3. 각 분할을 차례대로 검증데이터로 할당함
$\rightarrow$총 k번의 검증데이터의 성능측정이 가능하다는 의미

### 1-5. 중첩 교차검증
전체 자료를 학습데이터와 시험데이터로 분할 후 다시 학습데이터와 검증데이터로 구분하여 최적의 초모수 선택 및 모형의 최종성능을 계산하는 방법 [참고](https://thebook.io/080223/ch06/04/02/)

1. 전체 자료를 학습데이터와 시험데이터를 위한 $k_{1}$분할을 함
2. $k_{1}$개의 학습데이터 + 시험데이터 세트가 만들어짐
3. 세트 각각에 있는 학습데이터를 다시 동일하게 $k_{2}$분할을 통해 학습데이터와 검증데이터를 만듦



## 2. sklearn 모듈을 활용한 모형진단과 교차검증

학습데이터와 검증데이터로 분할하여 분석모델에서 사용하는 초모수들 중 최적의 초모수값 찾기

### 2-1. k 분할 교차검증 실시
sklearn.model_selection 모듈의 GridSearchCV().fit()함수 이용

> from sklearn.model_selection import GridSearchCV

> 객체명 = GridSearchCV(estimator=분석모델, param_grid=초모수값들, scoring='accuracy', cv=k분할 교차검증 수).fit(입력변수, 출력변수)

**estimator 옵션**
- 분석에 사용되는 분석모델의 함수명을 입력함
- 의사결정나무 분석이라면 sklearn.tree모듈의 DecisionTreeClassifier()함수를 초모수값 입력없이 사용
- SVM 분석이라면 sklearn.svm모듈의 SVC()함수를 초모수값 입력없이 사용

**parm_grid 옵션**
- 분석모델에서 사용되는 초모수 값들을 딕셔너리 형태로 초모수 옵션 이름과 대입할 값들을 리스트의 형태로 쌍으로 이루어 입력함.  
- 예를 들어 의사결정나무의 최대 깊이를 여러 개의 값을 두고 확인한다면 {'max_depth':[1,2,3,4,5]}와 같은 형태로 입력함.  
- 만약 초모수의 값들이 여러 개라고 한다면, 이러한 딕셔너리를 리스트 형태로 묶어서 param_grid 옵션에 입력함

**scoring 옵션**
- 최적의 초모수를 결정함에 있어 사용되는 통계량을 결정함   
- 기본적으로 분석모델에서 score()함수를 사용하였을 때 출력되는 값이 사용됨  

**cv 옵션**
- 몇 개의 폴드로 구분할지 결정

> 최적 정분류율: 객체명.best_score_

> 최적 초모수: 객체명.best_params_

### 2-2. 정분류율 확인
k분할 교차검증을 통해 구한 최적의 초모수값을 분석모델에 적용

> 객체명2 = 객체명1.best_estimator_.fit(입력변수, 출력변수)

**best_estimator_ 함수**
- 함수는 GridSearchCV()함수의 estimator옵션에 지정한 분석모델에 초모수 값을 지정한 형태

> 최적초모수적용 학습데이터 정분류율: 객체명2.score(입력변수, 출력변수)

### 2-3. 중첩 교차검증
최적의 초모수를 결정하고 모형을 비교하는 과정

> from sklearn.model_selection import StratifiedKFold

> inner_cv = StratifiedKFold(n_splits=inner loop횟수, shuffle=True, random_state=초기난수값)

> outer_cv = StratifiedKFold(n_splits=outer loop횟수, shuffle=True, random_state=초기난수값)

> from sklearn.model_selection import GridSearchCV

> 객체명1 = GridSearchCV(estimator=분석모델, param_grid=초모수값들, scoring='accuracy', cv=inner_cv)

**주의!**
- k분할 교차검증: GridSearchCV()함수에 바로 fit()함수 대입
- 중첩 교차검증: GridSearchCV()함수까지만 사용

$\rightarrow$ outer loop에서 데이터를 입력 후 나뉘어진 자료가 초모수 결정에 사용

> from sklearn.model_selection import cross_val_score

> 객체명2 = cross_val_score(객체명1, 입력변수, 출력변수, scoring='accuracy', cv=outer_cv)

$\rightarrow$ outer loop 마다 최적의 초모수를 결정하고, 이에 따른 정분류율을 계산하여 출력 (예. outer loop가 5회 수행되었다고 하면, 총 5개의 정분류율이 계산되어 출력됨.)

### 2-4. 의사결정나무와 SVM의 비교



In [1]:
import seaborn as sns

In [2]:
df = sns.load_dataset('iris')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train, test = train_test_split(df, test_size=0.3, random_state=1, stratify=df['species'])

In [6]:
y_train = train['species']
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]

y_test = test['species']
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]

#### 의사결정나무 모형

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [8]:
GS1_1 = GridSearchCV(estimator=DecisionTreeClassifier(random_state=1),
                    param_grid=[{'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['gini']},
                               {'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['entropy']}],
                    scoring='accuracy', cv=5).fit(X_train, y_train)

In [9]:
GS1_1.best_score_

0.9333333333333332

In [10]:
GS1_1.best_params_

{'criterion': 'gini', 'max_depth': 2}

In [11]:
clf1 = GS1_1.best_estimator_.fit(X_train, y_train)

In [12]:
clf1.score(X_train, y_train)

0.9523809523809523

In [13]:
clf1.score(X_test, y_test)

0.9555555555555556

#### SVM 모형

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [15]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

In [16]:
param_grid = [{'C':param_range, 'kernel':['linear']},
             {'C': param_range, 'gamma': param_range, 'kernel':['rbf']}]

In [17]:
GS2_1 = GridSearchCV(estimator=SVC(random_state=1), 
                    param_grid=param_grid,
                    scoring='accuracy', cv=5).fit(X_train, y_train)

In [18]:
GS2_1.best_score_

0.9619047619047618

In [19]:
GS2_1.best_params_

{'C': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}

In [20]:
clf2 = GS2_1.best_estimator_.fit(X_train, y_train)

In [21]:
clf2.score(X_train, y_train)

0.9809523809523809

In [22]:
clf2.score(X_test, y_test)

0.9777777777777777

SVM의 정분류율이 의사결정나무보다 높게 나타남  

다음은 중첩교차검증을 통해 최적의 초모수를 결정하고 모형을 평가하는 예시임.    

In [23]:
X=df[['sepal_length','sepal_width','petal_length','petal_width']]
y=df['species']

In [24]:
from sklearn.model_selection import StratifiedKFold

In [25]:
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [26]:
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [27]:
GS1_2 = GridSearchCV(estimator=DecisionTreeClassifier(random_state=1),
                    param_grid=[{'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['gini']},
                               {'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['entropy']}],
                    scoring='accuracy', cv=inner_cv)

In [28]:
from sklearn.model_selection import cross_val_score, cross_validate

In [29]:
scores = cross_val_score(GS1_2, X, y, scoring='accuracy', cv=outer_cv)

In [30]:
scores

array([0.93333333, 1.        , 0.93333333, 0.96666667, 0.9       ])

In [31]:
from numpy import mean, std

In [32]:
print('CV accuracy: %.3f +/- %.3f'%(mean(scores), std(scores)))

CV accuracy: 0.947 +/- 0.034


In [33]:
GS2_2 = GridSearchCV(estimator=SVC(random_state=1), 
                    param_grid=param_grid,
                    scoring='accuracy', cv=inner_cv)

In [34]:
scores = cross_val_score(GS2_2, X, y, scoring='accuracy', cv=outer_cv)

In [35]:
print(scores)

[1.         1.         0.93333333 1.         0.93333333]


In [36]:
print('CV accuracy: %.3f +/- %.3f'%(mean(scores), std(scores)))

CV accuracy: 0.973 +/- 0.033


SVM의 정분류율이 더 높은 것을 봐서 SVM이 더 정확한 예측을 하는 모형임을 알 수 있음

## 실습

### [과제15]
seaborn 모듈의 penguins 데이터에서 species를 예측하기 위해 bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g를 사용하여 k분할 교차검증을 통해 최적의 의사결정나무와 SVM의 모수를 탐색하고 두 모형을 비교할 것

In [37]:
df = sns.load_dataset('penguins')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [39]:
df = df.dropna(subset=['species','bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g'], how='any', axis=0)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            342 non-null    object 
 1   island             342 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 21.4+ KB


In [41]:
train, test = train_test_split(df, test_size=0.3, random_state=123, stratify=df['species'])

In [42]:
y_train = train['species']
X_train = train[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]

y_test = test['species']
X_test = test[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]

In [43]:
GS1_1 = GridSearchCV(estimator=DecisionTreeClassifier(random_state=1),
                    param_grid=[{'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['gini']},
                               {'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['entropy']}],
                    scoring='accuracy', cv=5).fit(X_train, y_train)

In [44]:
GS1_1.best_score_

0.9455673758865248

In [45]:
GS1_1.best_params_

{'criterion': 'entropy', 'max_depth': 4}

In [46]:
clf1 = GS1_1.best_estimator_.fit(X_train, y_train)

In [47]:
clf1.score(X_train, y_train)

0.9916317991631799

In [48]:
clf1.score(X_test, y_test)

0.9611650485436893

In [49]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

In [50]:
param_grid = [{'C':param_range, 'kernel':['linear']},
             {'C': param_range, 'gamma': param_range, 'kernel':['rbf']}]

In [51]:
GS2_1 = GridSearchCV(estimator=SVC(random_state=1), 
                    param_grid=param_grid,
                    scoring='accuracy', cv=5).fit(X_train, y_train)

In [52]:
GS2_1.best_score_

0.9875

In [53]:
GS2_1.best_params_

{'C': 1.0, 'kernel': 'linear'}

In [54]:
clf2 = GS2_1.best_estimator_.fit(X_train, y_train)

In [55]:
clf2.score(X_train, y_train)

0.99581589958159

In [56]:
clf2.score(X_test, y_test)

1.0

In [57]:
X=df[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
y=df['species']

In [58]:
from sklearn.model_selection import StratifiedKFold

In [59]:
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [60]:
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [61]:
GS1_2 = GridSearchCV(estimator=DecisionTreeClassifier(random_state=1),
                    param_grid=[{'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['gini']},
                               {'max_depth':[1,2,3,4,5,6,7,None], 'criterion':['entropy']}],
                    scoring='accuracy', cv=inner_cv)

In [62]:
from sklearn.model_selection import cross_val_score, cross_validate

In [63]:
scores = cross_val_score(GS1_2, X, y, scoring='accuracy', cv=outer_cv)

In [64]:
scores

array([0.97101449, 0.98550725, 0.98529412, 0.92647059, 0.91176471])

In [65]:
from numpy import mean, std

In [66]:
print('CV accuracy: %.3f +/- %.3f'%(mean(scores), std(scores)))

CV accuracy: 0.956 +/- 0.031


In [67]:
GS2_2 = GridSearchCV(estimator=SVC(random_state=1), 
                    param_grid=param_grid,
                    scoring='accuracy', cv=inner_cv)

In [68]:
scores = cross_val_score(GS2_2, X, y, scoring='accuracy', cv=outer_cv)

In [69]:
print(scores)

[0.97101449 0.98550725 1.         1.         0.97058824]


In [70]:
print('CV accuracy: %.3f +/- %.3f'%(mean(scores), std(scores)))

CV accuracy: 0.985 +/- 0.013


SVM의 정분률율이 더 높은 것을 확인